In [1]:
%pip install geemap

Note: you may need to restart the kernel to use updated packages.


Installing and Displaying GeeMap


In [24]:
import geemap
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Authenticating and Initialization of Earth Engine and setup of area of interest in the map


In [17]:
import ee
ee.Authenticate()
ee.Initialize()

#Defines the area of interest and Date range 
geometry = ee.Geometry.Polygon([
  [[-8.570156845611686,51.7292195887807], [-7.959042343658562,51.7292195887807], [-7.959042343658562,52.00904254772663], [-8.570156845611686,52.00904254772663],  [-8.570156845611686,51.7292195887807]]  
])

start_date = '2023'
end_date = '2024'



Image Collection and Filtering

In [36]:


# Image Collection and Filtering
sentinel_collection = ee.ImageCollection("COPERNICUS/S1_GRD") \
                        .filterDate(start_date, end_date) \
                        .filterBounds(geometry)

# Check if the collection has any images
if sentinel_collection.size().getInfo() > 0:
    # Create a composite using the 'VV' and 'VH' bands
    sentinel_composite = sentinel_collection.select(['VV', 'VH']).median()

    # Verify the composite
    print(sentinel_composite.getInfo())

    # Get the region coordinates for export
    region = geometry.coordinates().getInfo()

    # Export the composite image to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=sentinel_composite,
        description='Sentinel_Composite',
        scale=30,
        region=region,
        fileFormat='GeoTIFF'
    )
    task.start()
else:
    print("No images found for this date range and region.")



{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [37]:
print(task.status())

{'state': 'RUNNING', 'description': 'Sentinel_Composite', 'priority': 100, 'creation_timestamp_ms': 1730207634685, 'update_timestamp_ms': 1730207638053, 'start_timestamp_ms': 1730207638016, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GSEGPRC2C6XL4XQUOA2AUT7D', 'name': 'projects/312891332570/operations/GSEGPRC2C6XL4XQUOA2AUT7D'}


In [44]:
import rasterio
import matplotlib.pyplot as plt

# Open the GeoTIFF file
file_path = 'C:/Users/Neo/Desktop/FYP/geeimages/Sentinel_Composite.tif'
with rasterio.open(file_path) as src:
    img = src.read([1, 2, 3])  # Use the first three bands (if applicable) for RGB

# Plot the image
plt.imshow(img.transpose(1, 2, 0))  # Transpose for correct display
plt.axis('off')  # Turn off axis
plt.show()


ModuleNotFoundError: No module named 'rasterio'